<a href="https://colab.research.google.com/github/HAL22/Kaggle-Competitions/blob/Quora-Insincere-Questions-Classification/Quora_Insincere_Questions_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install kaggle
import pandas as pd
!pip install datasets transformers[sentencepiece]
!apt install git-lfs
from datasets import load_dataset
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer)
import torch
from transformers import pipeline
from datasets import Features, Value, ClassLabel

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [3]:
!git config --global user.email "thethelafaltein@gmail.com"
!git config --global user.name "HAL22"

In [4]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [5]:
!kaggle competitions download -c quora-insincere-questions-classification

100% 6.03G/6.03G [04:42<00:00, 25.4MB/s]
100% 6.03G/6.03G [04:42<00:00, 22.9MB/s]


In [6]:
!unzip /content/quora-insincere-questions-classification.zip

Archive:  /content/quora-insincere-questions-classification.zip
  inflating: embeddings.zip          
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [7]:
df_train = pd.read_csv("train.csv")
df_validation = pd.read_csv("test.csv")
df_train = df_train.set_index('qid')
#index_col removes the index column

In [8]:
df_train.head()

,question_text,target
qid,,
00002165364db923c7e6,How did Quebec nationalists see their province...,0
000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [9]:
df_train.rename(columns = {'question_text':'text', 'target':'label'}, inplace = True)

In [14]:
#df_train.set_index('text',inplace=True)

In [10]:
df_train.head()

,text,label
qid,,
00002165364db923c7e6,How did Quebec nationalists see their province...,0
000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [11]:
df_train = df_train[['text','label']]

In [12]:
df_train.head()

,text,label
qid,,
00002165364db923c7e6,How did Quebec nationalists see their province...,0
000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [13]:
df_train.reset_index(drop=True, inplace=True)

In [14]:
df_train.head()

,text,label
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


In [15]:
df_train, df_test = train_test_split(df_train, test_size=0.2)

In [16]:
df_train.head()

,text,label
181845,What are gerunds used for in Turkish? What wou...,0
1268379,Do you girls look different without your makeu...,0
158425,Should I go for MS in CS or MS in Data Science...,0
743879,Which one do you select be an actor/ actress o...,0
98923,Does it bode well for the Wheel of Time TV ser...,0


In [17]:
df_validation = df_validation[['question_text']]
df_validation.rename(columns = {'question_text':'text'}, inplace = True)
df_validation.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text
0,Why do so many women become so rude and arroga...
1,When should I apply for RV college of engineer...
2,What is it really like to be a nurse practitio...
3,Who are entrepreneurs?
4,Is education really making good people nowadays?


In [18]:
train = Dataset.from_pandas(df_train)
test = Dataset.from_pandas(df_test)
validation = Dataset.from_pandas(df_validation)

In [19]:
dataset = DatasetDict()
dataset['train'] = train
dataset['test'] = test
dataset['validation'] = validation

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 1044897
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 261225
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 375806
    })
})

In [21]:
model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [22]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [23]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1045 [00:00<?, ?ba/s]

  0%|          | 0/262 [00:00<?, ?ba/s]

  0%|          | 0/376 [00:00<?, ?ba/s]

In [24]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
model_bert = AutoModelForSequenceClassification.from_pretrained(model)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [26]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=0.22,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [27]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Step,Training Loss
500,0.160900
1000,0.135600
1500,0.127900
2000,0.128400
2500,0.123200
3000,0.125500
3500,0.126500
4000,0.117200
4500,0.111600
5000,0.110500


Step,Training Loss
500,0.160900
1000,0.135600
1500,0.127900
2000,0.128400
2500,0.123200
3000,0.125500
3500,0.126500
4000,0.117200
4500,0.111600
5000,0.110500


TrainOutput(global_step=14368, training_loss=0.1167563409741578, metrics={'train_runtime': 1179.5745, 'train_samples_per_second': 194.882, 'train_steps_per_second': 12.181, 'total_flos': 2305483304240064.0, 'train_loss': 0.1167563409741578, 'epoch': 0.22})

In [33]:
predictions = trainer.predict(tokenized_dataset["validation"])

In [35]:
predictions.predictions

array([[-0.7244524,  0.6915656],
       [ 4.1632857, -3.2756908],
       [ 4.0932717, -3.255887 ],
       ...,
       [ 3.9493449, -3.167127 ],
       [ 4.136252 , -3.2442894],
       [ 2.1638994, -1.5378865]], dtype=float32)

In [36]:
import numpy as np


In [37]:
pred = np.argmax(predictions.predictions, axis=-1)

In [38]:
pred[0]

1

In [40]:
df_validation = pd.read_csv("test.csv")
df_validation['prediction'] = pred

In [41]:
df_validation.head()

,qid,question_text,prediction
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...,1
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...,0
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...,0
3,000086e4b7e1c7146103,Who are entrepreneurs?,0
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?,0


In [42]:
df_validation = df_validation[['qid','prediction']]

In [43]:
df_validation.head()

,qid,prediction
0,0000163e3ea7c7a74cd7,1
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0


In [44]:
df_validation.to_csv('submission.csv')